The main purpose of the ocean module is to 

 * collect all environmental data in one place ('one module to rule them all')
 * provide a high-level interface for loading and querying/interpolating environmental data
 
In particular, the ocean module should allow the user to

 * specify data sources via simple keys (e.g. 'HYCOM')
 * load data for a specific (rectangular) geographic region and time range
 * query (interpolated) data on any given grid
 
The current implementation of the ocean module already supports some of these uses.

The ocean module is already being used by the kadlu.sound.sound_propagation module (implementation almost completed) and the kadlu.sound.geophony module (implementation partially completed). 

# 1. Initialization

In the most common scenario, the user initializes an instance of the ocean module by simply specifying the data sources. For example,

In [18]:
from importlib import reload
from kadlu.geospatial import ocean as oceanmodule
from datetime import datetime
from kadlu.geospatial.data_sources import hycom

In [8]:
reload(oceanmodule)
Ocean = oceanmodule.Ocean

ocean = Ocean(bathy='CHS', temp='HYCOM', salinity='HYCOM', wave='WWIII')

In a second, less common, but also relevant, scenario, the user provides the data in the form of numpy arrays. For example,

In [9]:
import numpy as np 

lats = np.linspace(45, 46, 10)   # longitudes
lons = np.linspace(120, 122, 20) # latitudes
depths = -100 * np.ones(shape=(10,20))   # flat bathymetry (100 m depth everywhere)

ocean = Ocean(bathy=(depths, lats, lons), temp='HYCOM', salinity='HYCOM', wave='WWIII')

Finally, for testing purposes, it may convenient to allow the user to specify a constant value for any of the inputs. For example, 

In [10]:
ocean = Ocean(bathy=-100, temp='HYCOM', salinity='HYCOM', wave='WWIII')

# 1.5 Fetching

In [25]:
south, west = 44,-60
north, east = 45,-59
top, bottom = 0,5000
start=datetime(2015, 10, 1)
end=datetime(2015, 10, 1, 12)

In [26]:
hycom.Hycom().fetch_salinity(south=south, north=north, west=west, east=east, 
                             start=start, end=end, top=top, bottom=bottom)
hycom.Hycom().fetch_temp(south=south, north=north, west=west, east=east,
                         start=start, end=end, top=top, bottom=bottom)

downloading 72800 salinity values from hycom...
downloaded 75.9Kb in 111.259s. parsed and inserted 1820 rows in 0.512s
36985 null values removed, 33995 duplicate rows ignored

downloading 72800 water_temp values from hycom...
downloaded 77.6Kb in 121.156s. parsed and inserted 1820 rows in 0.514s
36985 null values removed, 33995 duplicate rows ignored



# 2. Loading

In the most common scenario, the user simply loads all data for a given region and time range, as follows

In [27]:
ocean.load(south=south, north=north, west=west, east=east, 
           start=start, end=end, top=top, bottom=bottom) #, time=None)

AssertionError: values must be a 3d array

The current implementation only supports a single time value. This should be changed to also provide support for a range of times, i.e., start and end time. Also note, that automatic fetching/loading of HYCOM and WWIII data has not yet been implemented.

### To-do's:
* Implement support for time range (start and end times) in load method
* Implement loading of HYCOM and WWIII data (as well as other sources)

In a second, less common, but also relevant, scenario, the user will want to load only one data variable, e.g., bathymetry. For this, the ocean module has specific load methods for each of the four data types (bathy, temp, salinity, wave). For example,

In [5]:
ocean.load_bathy(south=44, north=45, west=120, east=122)

# Querying/interpolating ('get' methods)

Currently, the ocean module has the following 'get' methods:

 * bathy 
 * bathy_gradient
 * temp
 * salinity
 * wave
 
These methods allow computation of the desired variable at a single point, on a set of points, or on a regular grid, specified either in planar (x-y) or spherical (lat-lon) geometry.

The methods make use of the kadlu.geospatial.interpolation module to perform these tasks.

For example, we can query the bathymetry at two points, like this

In [6]:
bathy_values = ocean.bathy(x=[0,1000], y=[0,500])
print(bathy_values)

[-100. -100.]


### To-do's: 
* There is currently only a single generic 'get' method for wave data, even though there are multiple variables (height, period, etc). We should either implement a method for each wave variable, or allow the user to specify the variable as an input argument to the 'get' method. 